In [4]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from Data import Data
from Unet import Unet

In [5]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def dice_sim(pred, truth):
    epsilon = 1e-8
    num_batches = pred.size(0)
    m1 = pred.view(num_batches, -1).bool()
    m2 = truth.view(num_batches, -1).bool()

    intersection = torch.logical_and(m1, m2).sum(dim=1)
    return (((2. * intersection + epsilon) / (m1.sum(dim=1) + m2.sum(dim=1) + epsilon)).sum(dim=0))/2

train = Data('train', None)
val = Data('validate', None)
test = Data('test', None)

batch_size = 2

train_generator = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True, num_workers=0)
val_generator = torch.utils.data.DataLoader(val, batch_size=2, shuffle=True, num_workers=0)
test_generator = torch.utils.data.DataLoader(test, batch_size=2, shuffle=True, num_workers=0)


In [6]:

model = Unet().to(device)
#model = torch.load(dir+"model", map_location=torch.device("cpu")) To load pretrained model

loss_fn = nn.BCELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, min_lr=1e-9)
train_size = len(train)
val_size = len(val)


In [18]:
num_epochs = 2


In [19]:
df = pd.DataFrame([],
                  columns=['Epoch','TrainLoss', 'TrainAcc', 'ValLoss', 'ValAcc'])

In [20]:
df

,Epoch,TrainLoss,TrainAcc,ValLoss,ValAcc


In [21]:
from datetime import datetime
print(datetime.now(tz=None))


2021-11-28 22:34:21.620667


In [ ]:
for epoch in range(4,30):
    print(datetime.now(tz=None))
    print(df)

    running_loss = 0.0
    val_loss = 0.0
    val_acc = 0.0
    train_acc = 0.0

    # Training
    for i, data in enumerate(train_generator):
        image, truth = data
        truth = truth/255
        predictions = model(image)
        loss = loss_fn(predictions, truth)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_acc += dice_sim(predictions, truth)*100
        #print("Running iteration i=", i, " running loss=", running_loss)
        
    # Validation
    with torch.no_grad():
        for i, data in enumerate(val_generator):
            image, truth = data
            truth = truth/255
            predictions = model(image)
            predictions = torch.round(predictions) # Rounding the predictions for classification
            loss = loss_fn(predictions, truth)
            val_loss += loss.item()
            val_acc += dice_sim(predictions, truth)*100

    epoch_train_loss = running_loss / (train_size//batch_size+1)
    epoch_val_loss = val_loss / (val_size//batch_size+1)
    epoch_val_acc = val_acc / (val_size//batch_size+1)
    epoch_train_acc = train_acc / (train_size//batch_size+1)
    scheduler.step(epoch_val_loss) # LR Scheduler
    
    torch.save(model.state_dict(), f"/home/ubuntu/aae/models/aae-ep-{epoch}")
    df.loc[epoch] = [epoch, epoch_train_loss, epoch_train_acc.item(), epoch_val_loss, epoch_val_acc.item()]
    print(df)


2021-11-28 22:58:54.340314
   Epoch  TrainLoss   TrainAcc   ValLoss     ValAcc
0    0.0   0.139771  45.618732  5.711706  68.822235
1    1.0   0.114984  45.618744  2.718837  74.948235
2    2.0   0.102017  45.618759  2.721866  75.372414
3    3.0   0.093084  45.618740  3.526864  73.650551
   Epoch  TrainLoss   TrainAcc   ValLoss     ValAcc
0    0.0   0.139771  45.618732  5.711706  68.822235
1    1.0   0.114984  45.618744  2.718837  74.948235
2    2.0   0.102017  45.618759  2.721866  75.372414
3    3.0   0.093084  45.618740  3.526864  73.650551
4    4.0   0.085401  45.618759  2.122324  76.152641
2021-11-28 23:04:14.777556
   Epoch  TrainLoss   TrainAcc   ValLoss     ValAcc
0    0.0   0.139771  45.618732  5.711706  68.822235
1    1.0   0.114984  45.618744  2.718837  74.948235
2    2.0   0.102017  45.618759  2.721866  75.372414
3    3.0   0.093084  45.618740  3.526864  73.650551
4    4.0   0.085401  45.618759  2.122324  76.152641


In [25]:
model.load_state_dict(torch.load("/home/ubuntu/aae/models/aae-ep-3", map_location="cuda:0"))  
model.to(device)

Unet(
  (inc): start_conv(
    (conv): res_conv(
      (conv1): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.5, inplace=False)
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (down1): down_conv(
    (conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): res_conv(
        (conv1): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReL